In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [125]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using LinearMaps
using IterativeSolvers
using BenchmarkTools
using LoopVectorization

┌ Info: Precompiling KrylovKit [0b1a1467-8014-51b9-945f-bf0ae24f4b77]
└ @ Base loading.jl:1278


In [150]:
using LinearAlgebra
using LinearMaps
using IterativeSolvers

Nx = 20
Ny = 10

A = randn(Ny, Nx)
Ne = 500
X = randn(Nx, Ne)

function g!(Ny, Nx, Ne, vout, vin)
    fill!(vout, 0.0)
    
    @inbounds for j=1:Ne
        # chunk of dimension Nx
        vj = view(vin, (j-1)*Nx+1:j*Nx)
        # A will be the Jacobian of the function f evaluated at x^i
        mul!(vout, A, vj, 1.0, 1.0)
    end
    vout .*= 1/sqrt(Ne)
end

sqrtCx = LinearMap((vout, vin) -> g!(Nx, Ne, vout, vin), Ny, Nx*Ne; ismutating = true)

cache = ones(Nx, Ne)
@btime 1/sqrt(Ne)*sum(A*cache;dims = 2)

v = ones(Nx*Ne);
@btime sqrtCx*v

@show norm(1/sqrt(Ne)*sum(A*ones(Nx, Ne);dims = 2)-sqrtCx*v)

  12.948 μs (10 allocations: 39.56 KiB)
  39.838 μs (501 allocations: 23.59 KiB)
norm((1 / sqrt(Ne)) * sum(A * ones(Nx, Ne); dims = 2) - sqrtCx * v) = 5.097638705940257e-12


5.097638705940257e-12

In [157]:
sqrtCx'*ones(Ny)

LoadError: [91mtranspose not implemented for 10×10000 LinearMaps.FunctionMap{Float64}(#59; ismutating=true, issymmetric=false, ishermitian=false, isposdef=false)[39m

In [156]:
svdl(sqrtCx; nsv = 5, vecs = :right)

LoadError: [91mtranspose not implemented for 10×10000 LinearMaps.FunctionMap{Float64}(#59; ismutating=true, issymmetric=false, ishermitian=false, isposdef=false)[39m

In [148]:
?svdl

search: svdl svdvals svdvals! svdsolve svd! svd SVD GeneralizedSVD isvalid



```
svdl(A) -> Σ, L, [history]
```

Compute some singular values (and optionally vectors) using Golub-Kahan-Lanczos bidiagonalization [^Golub1965] with thick restarting [^Wu2000].

If `log` is set to `true` is given, method will output a tuple `X, L, ch`. Where `ch` is a `ConvergenceHistory` object. Otherwise it will only return `X, L`.

# Arguments

  * `A` : The matrix or matrix-like object whose singular values are desired.

## Keywords

  * `nsv::Int = 6`: number of singular values requested;
  * `v0 = random unit vector`: starting guess vector in the domain of `A`. The length of `q` should be the number of columns in `A`;
  * `k::Int = 2nsv`: maximum number of Lanczos vectors to compute before restarting;
  * `j::Int = nsv`: number of vectors to keep at the end of the restart. We don't recommend j < nsv;
  * `maxiter::Int = minimum(size(A))`: maximum number of iterations to run;
  * `verbose::Bool = false`: print information at each iteration;
  * `tol::Real = √eps()`: maximum absolute error in each desired singular value;
  * `reltol::Real=√eps()`: maximum error in each desired singular value relative to the estimated norm of the input matrix;
  * `method::Symbol=:ritz`: restarting algorithm to use. Valid choices are:

    1. `:ritz`: Thick restart with Ritz values [Wu2000].
    2. `:harmonic`: Restart with harmonic Ritz values [Baglama2005].
  * `vecs::Symbol = :none`: singular vectors to return.

    1. `:both`: Both left and right singular vectors are returned.
    2. `:left`: Only the left singular vectors are returned.
    3. `:right`: Only the right singular vectors are returned.
    4. `:none`: No singular vectors are returned.
  * `dolock::Bool=false`: If `true`, locks converged Ritz values, removing them from the Krylov subspace being searched in the next macroiteration;
  * `log::Bool = false`: output an extra element of type `ConvergenceHistory` containing extra information of the method execution.

# Return values

**if `log` is `false`**

  * `Σ`: list of the desired singular values if `vecs == :none` (the default), otherwise returns an `SVD` object with the desired singular vectors filled in;
  * `L`: computed partial factorizations of A.

**if `log` is `true`**

  * `Σ`: list of the desired singular values if `vecs == :none` (the default),

otherwise returns an `SVD` object with the desired singular vectors filled in;

  * `L`: computed partial factorizations of A;
  * `history`: convergence history.

**ConvergenceHistory keys**

  * `:betas` => `betas`: The history of the computed betas.
  * `:Bs` => `Bs`: The history of the computed projected matrices.
  * `:ritz` => `ritzvalhist`: Ritz values computed at each iteration.
  * `:conv` => `convhist`: Convergence data.

[^Golub1965]: Golub, Gene, and William Kahan. "Calculating the singular values and pseudo-inverse of a matrix." Journal of the Society for Industrial and Applied Mathematics, Series B: Numerical Analysis 2.2 (1965): 205-224.

[^Wu2000]: Wu, Kesheng, and Horst Simon. "Thick-restart Lanczos method for large symmetric eigenvalue problems." SIAM Journal on Matrix Analysis and Applications 22.2 (2000): 602-616.
